<img style="float: right; margin: 0px 0px 15px 15px;" src="https://www.carrerasenlinea.mx/logos/original/logo-instituto-tecnologico-y-de-estudios-superiores-de-occidente.png" width="150px" height="100px" />

# Laboratorio 3: Behavioral Economics en Trading.


- El propósito: Que utilices la información histórica de la cuenta de trading manual y, mediante la formulación y programación de 3 "Sesgos cognitivos", realices una evaluación del desempeño de la cuenta utilizando el marco de "Behavioral economics". Con la finalidad de inferir si al momento de tomar las decisiones que se tomaron en esa cuenta, se estuvo incurriendo en esos "Sesgos cognitivos" y su impacto en el resultado final.


Equipo 5:

* Luis Ontaneda 
* Alejandro Preciado
* Jose Antonio Quintero
* Ana Paula Rangel


## Sesgo cognitivo elegido: Overconfidence

Los sesgos cognitivos se basan en diferencias a la hora de interpretar la información.

Este sesgo consiste en la creer tener mas información de la seguridad con la que se toma una decisión, basada en una decisión propia del pasado. Esto pasa por que lleva a creer que tenemos más información que el resto para la toma de decisiones, lo que lleva a pensar que la probabilidad de fracaso es menor a la real.

En el ambito financiero una persona puede llegar a sobreestimar las ganancias esperadas.

Ejemplo genérico: 

    •	Martingala

Aplicación a trading:

    •	Repetir posiciones en mayor cantidad 



In [3]:
#se importan las librerias
import pandas as pd
import numpy as np
import json #leer el archivo
from datetime import datetime #Para separar las fechas por semanas 
import pandas as pd
import matplotlib.pyplot as plt #para las graficas
import matplotlib
import plotly.express as px
import plotly.graph_objs as go


In [4]:

def f_datosent(P0_archivo):
    """
    :param p0_archivo: archivo de entrada
    :return resultado
    
    debbuging
    p0_archivo = 'archivo_tradeview_2.xlsx'
    
    """
    import pandas as pd
    import numpy as np
    import json
    
    file_type = P0_archivo[-4:]
    # Opciones para visualizar data frames en consola
    pd.set_option('display.max_rows',5000)
    pd.set_option('display.max_columns',500)
    pd.set_option('display.width',1000)
    
    # Ingreso de datos de entrada: Leer archivo JSON
    archivo = P0_archivo
    
    if file_type == 'json':
        with open(archivo, encoding='utf-8-sig') as json_file:
            datos_2 = json.load(json_file)['data']
            
        # Seleccionar la seccion de 'closedTransactions' que es la de interes
        df_datos = pd.DataFrame(datos_2['closedTransactions']['list'])
        
        # Seleccionar solo los renglones de operaciones
        df_datos = df_datos[(df_datos['type'] == 'buy') | (df_datos['type'] == 'sell') | (df_datos['type'] == 's/l') |
                (df_datos['type'] == 't/p')]
        
        # Resetear index para tener todos los numeros de indice completos
        df_datos = df_datos.reset_index()
        
        # Eliminar columnas 'info' e 'index'
        df_datos = df_datos.drop(['info', 'index'], 1)
        
        # Renombrar columnas
        df_datos.rename(columns = {'SL': 'sl', 'TP':'tp', 'price':'openPrice', 'price2':'closePrice', 'item':'Instrument'},
                        inplace=True)
        return df_datos
    elif file_type == 'xlsx':
        df_datos = pd.read_excel(archivo, sheet_name = 'Hoja1', col_header=0)
        return df_datos

In [5]:
#import untitled1
#f_datosent(P0_archivo="archivo_tradeview_0.json")

In [6]:
def f_bf_overconfidence(file):
    from datetime import datetime #Numeros a formato de fecha
    import pandas as pd
    
    
    df=f_datosent(file) #archivo
    file_type = file[-4:]
    
    
    if file_type == 'json': ## función sí los datos vienen en formato json
        
        df["size"]=df["size"].apply(pd.to_numeric) #convertir datos a numericos
        df["profit"]=df["profit"].apply(pd.to_numeric)
        
    

        df["closeTime"] = df["closeTime"].apply(lambda x: datetime.strptime(x, '%Y.%m.%d %H:%M:%S')) #Convertimos los numeros en formato de fecha
        weekly = df.groupby(pd.Grouper(key='closeTime', freq='W'))
        weekly = [weekly.get_group(x) for x in weekly.groups] #agrupamos por semana

        oc = 0
        w = []
        dfs = []
        week= []
        for z in range(len(weekly)):
            wtype = weekly[z].groupby(["type","Instrument"]) ## agrupamos por semana nuestras variables a usar
            wtype = [wtype.get_group(y) for y in wtype.groups] 
            for x in range(len(wtype)):
                for i in range(1,len(wtype[x])):
                    if wtype[x].reset_index().loc[i-1,"profit"] > 0: ## condicion para observar si hubo profit y asi el sujeto es mas propenso a hacer el sesgo
                        if wtype[x].reset_index().loc[i-1,"size"] < wtype[x].reset_index().loc[i,"size"]: ## condición que muestra que el volumen ha subido ya que registro ganancia en ese mismo instrumento
                            oc += 1
                            w.append(z)
                            week.append(wtype[x]["closeTime"].iloc[0])
                            dfs.append(wtype[x]) 
        dfs = pd.concat(dfs)           
        plt.hist(week)
        plt.title('Sesgo de sobreconfianza', fontsize = "16")
        plt.xticks(rotation=30, color = "red")
        plt.xlabel("fechas en que realizó el sesgo")
        plt.ylabel("veces que realizó el sesgo")
        plt.show()
        

        return dfs
        
    
    
    
    elif file_type == 'xlsx': ## función sí los datos vienen en formato excel

        df["closeTime"] = df["closeTime"].apply(lambda x: datetime.strptime(x, '%Y.%m.%d %H:%M:%S'))
        weekly = df.groupby(pd.Grouper(key='closeTime', freq='W'))
        weekly = [weekly.get_group(x) for x in weekly.groups]

        oc = 0
        w = []
        dfs = []
        week=[]
        for z in range(len(weekly)):
            wtype = weekly[z].groupby(["Type","Symbol"]) ## agrupamos por semana nuestras variables a usar
            wtype = [wtype.get_group(y) for y in wtype.groups] 
            for x in range(len(wtype)):
                for i in range(1,len(wtype[x])):
                    if wtype[x].reset_index().loc[i-1,"Profit"] > 0: ## condicion para observar si hubo profit y asi el sujeto es mas propenso a hacer el sesgo
                        if wtype[x].reset_index().loc[i-1,"Size"] < wtype[x].reset_index().loc[i,"Size"]: ## condición que muestra que el volumen ha subido ya que registro ganancia en ese mismo instrumento
                            oc += 1
                            w.append(z)
                            week.append(wtype[x]["closeTime"].iloc[0])
                            dfs.append(wtype[x])
    dfs = pd.concat(dfs)
    plt.hist(week)
    plt.title('Sesgo de sobreconfianza', fontsize = "16")
    plt.xticks(rotation=30, color = "red")
    plt.xlabel("fechas en que realizó el sesgo")
    plt.ylabel("veces que realizó el sesgo")
    #plt.ylim(0,5) No es necesario hacer el eje de la y más grande debido a que va contando los "overconfidence" de 1 en 1 aunque sean seguidos
    plt.show()

    return (dfs)

In [56]:
def f_bf_overconfidence(file):
    from datetime import datetime #Numeros a formato de fecha
    import pandas as pd
    
    
    df=f_datosent(file) #archivo
    file_type = file[-4:]
    
    
    if file_type == 'json': ## función sí los datos vienen en formato json
        
        df["size"]=df["size"].apply(pd.to_numeric) #convertir datos a numericos
        df["profit"]=df["profit"].apply(pd.to_numeric)
        
    

        df["closeTime"] = df["closeTime"].apply(lambda x: datetime.strptime(x, '%Y.%m.%d %H:%M:%S')) #Convertimos los numeros en formato de fecha
        weekly = df.groupby(pd.Grouper(key='closeTime', freq='W'))
        weekly = [weekly.get_group(x) for x in weekly.groups] #agrupamos por semana

        oc = 0
        w = []
        dfs = []
        c = 0
        week= []
        for z in range(len(weekly)):
            wtype = weekly[z].groupby(["type","Instrument"]) ## agrupamos por semana nuestras variables a usar
            wtype = [wtype.get_group(y) for y in wtype.groups] 
            for x in range(len(wtype)):
                for i in range(1,len(wtype[x])):
                    c += 1
                    if wtype[x].reset_index().loc[i-1,"profit"] > 0: ## condicion para observar si hubo profit y asi el sujeto es mas propenso a hacer el sesgo
                        if wtype[x].reset_index().loc[i-1,"size"] < wtype[x].reset_index().loc[i,"size"]: ## condición que muestra que el volumen ha subido ya que registro ganancia en ese mismo instrumento
                            oc += 1
                            w.append(z)
                            week.append(wtype[x]["closeTime"].iloc[0])
                            dfs.append(wtype[x]) 
        dfs = pd.concat(dfs)           
        week = pd.DataFrame(week,columns=["closeTime"])
        fig = px.histogram(week, x="closeTime")
        p = oc/c
        

        return {"Dataframe": dfs, "Grafica": fig, "Explicacion": "Nuestro sesgo es el de Overconfidence el cual consiste en que gracias al exito de una persona en transacciones pasadas esta misma piensa que su habilidad es infalible por lo que empieza a tomar mas riesgo",
                "Porcentaje": "{:.1%}".format(p)}
                
    
    elif file_type == 'xlsx': ## función sí los datos vienen en formato excel

        df["closeTime"] = df["closeTime"].apply(lambda x: datetime.strptime(x, '%Y.%m.%d %H:%M:%S'))
        weekly = df.groupby(pd.Grouper(key='closeTime', freq='W'))
        weekly = [weekly.get_group(x) for x in weekly.groups]

        oc = 0
        c = 0
        w = []
        dfs = []
        week=[]
        for z in range(len(weekly)):
            wtype = weekly[z].groupby(["Type","Symbol"]) ## agrupamos por semana nuestras variables a usar
            wtype = [wtype.get_group(y) for y in wtype.groups] 
            for x in range(len(wtype)):
                for i in range(1,len(wtype[x])):
                    c += 1
                    if wtype[x].reset_index().loc[i-1,"Profit"] > 0: ## condicion para observar si hubo profit y asi el sujeto es mas propenso a hacer el sesgo
                        if wtype[x].reset_index().loc[i-1,"Size"] < wtype[x].reset_index().loc[i,"Size"]: ## condición que muestra que el volumen ha subido ya que registro ganancia en ese mismo instrumento
                            oc += 1
                            w.append(z)
                            week.append(wtype[x]["closeTime"].iloc[0])
                            dfs.append(wtype[x])
    dfs = pd.concat(dfs)
    week = pd.DataFrame(week,columns=["closeTime"])
    fig = px.histogram(week, x="closeTime")
    p = oc/c
    
    return {"Dataframe": dfs, "Grafica": fig, "Explicacion": "Nuestro sesgo es el de Overconfidence el cual consiste en que gracias al exito de una persona en transacciones pasadas esta misma piensa que su habilidad es infalible por lo que empieza a tomar mas riesgo",
                "Porcentaje": "{:.1%}".format(p)}

In [60]:
f_bf_overconfidence("archivo_tradeview_2.xlsx").get("Grafica")

In [ ]:
"{:.1%}".format(p)